## 라이브러리 및 함수 선언

In [1]:
import os
import json
from datetime import datetime

import h5py
import numpy as np
import pandas as pd

In [2]:
def get_file_list(file_path, extension):
    file_list = []
    for path, dirs, files in os.walk(file_path):
        path = path.replace("\\", "/")
        for file in files:
            if os.path.splitext(file)[1].lower() == extension:
                file_list.append("{}/{}".format(path, file))
    
    return file_list

In [3]:
def dummies_columns(columns_name, dataframe):
    dummies_one_hot = pd.get_dummies(dataframe[columns_name], prefix=columns_name)
    dataframe = pd.concat([pd.DataFrame(dataframe), dummies_one_hot], axis=1)
    
    return list(dummies_one_hot), dataframe

In [4]:
def columns_add(columns_dict, column_name, column_list, dataframe):
    columns = columns_dict[column_name]
    columns_add = list(set(columns) - set(column_list))
    
    dataframe[columns_add] = 0
    
    return dataframe, columns

## 앨범 메타데이터 가져오기

In [5]:
music_info_df = pd.read_csv("../data/meta/bugs_music_info.csv").rename(columns={"장르": "앨범 장르"})
music_info_df = music_info_df[music_info_df["발매일"].notnull()].reset_index(drop=True)
music_info_df["발매일_datetime"] = music_info_df["발매일"].apply(
    lambda x: datetime.strptime(x, "%Y.%m") if len(x) < 10 else datetime.strptime(x, "%Y.%m.%d"))
music_info_df

,sing_id,subject,singer,앨범 종류,발매일,앨범 장르,스타일,기획사,유통사,발매일_datetime
0,89688529,십이월 이십오일의 고백,정승환,싱글,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05
1,89613039,Blueming,아이유 (IU),EP(미니),2019.11.18,댄스/팝,발라드,카카오엔터테인먼트,카카오엔터테인먼트,2019-11-18
2,89682091,아마두 (Feat. 우원재 & 김효은 & 넉살 & Huckleberry P),염따 & 딥플로우 & 팔로알토 (Paloalto) & The Quiett & 사이먼...,싱글,2019.12.03,랩/힙합,랩,"AOMG, 딩고뮤직, Hi-Lite Records, ILLIONAIRE RECORD...",Dreamus,2019-12-03
3,89678583,나의 오랜 연인에게,다비치,싱글,2019.12.03,발라드,발라드,WAKEONE,지니뮤직,2019-12-03
4,89602794,Into the Unknown,Idina Menzel & AURORA,싱글,2019.12.13,팝,팝,Sing2Music Productions Pty Ltd,FUGA,2019-12-13
...,...,...,...,...,...,...,...,...,...,...
6594,94618083,Easy On Me,Adele,싱글,2021.10.14,팝,소울 팝,Columbia,Sony Music,2021-10-14
6595,94609018,우리의 이야기,멜로망스 (MeloMance),싱글,2021.10.15,인디,인디 팝,Stone Music Entertainment,지니뮤직,2021-10-15
6596,94611799,To You (Prod. by 정키),HYNN(박혜원) & 정키,EP(미니),2021.10.15,발라드,발라드,뉴오더 엔터테인먼트,카카오엔터테인먼트,2021-10-15
6597,94611800,남자가 되면,HYNN (박혜원),EP(미니),2021.10.15,발라드,발라드,뉴오더 엔터테인먼트,카카오엔터테인먼트,2021-10-15


## 앨범 차트 데이터 가져오기

In [6]:
chart_one_df = pd.read_csv("../data/split_chart/chart_one.csv", encoding="cp949")
chart_one_df

,date,rank,change,sing_id,subject,singer,rank_out_term,play_in_term,listen_in_term,play_total,listen_total
0,2019/12/08/11,1,-,89688529,십이월 이십오일의 고백,정승환,0,NaN,NaN,1939149,717398
1,2019/12/08/12,1,-,89688529,십이월 이십오일의 고백,정승환,1,30490.0,6547.0,1969639,723945
2,2019/12/08/13,1,-,89688529,십이월 이십오일의 고백,정승환,1,32874.0,7029.0,2002513,730974
3,2019/12/08/14,1,-,89688529,십이월 이십오일의 고백,정승환,1,34436.0,7139.0,2036949,738113
4,2019/12/08/15,1,-,89688529,십이월 이십오일의 고백,정승환,1,35033.0,7145.0,2071982,745258
...,...,...,...,...,...,...,...,...,...,...,...
2811,2021/01/04/15,224,NEW,89688529,십이월 이십오일의 고백,정승환,96,35781.0,1476.0,17589520,1511146
2812,2021/01/05/9,227,NEW,89688529,십이월 이십오일의 고백,정승환,18,7471.0,377.0,17596991,1511523
2813,2021/01/05/17,218,NEW,89688529,십이월 이십오일의 고백,정승환,8,4192.0,202.0,17601183,1511725
2814,2021/01/06/9,238,NEW,89688529,십이월 이십오일의 고백,정승환,16,6208.0,263.0,17607391,1511988


## 데이터 병합 (앨범 메타 + 차트)

In [7]:
chart_info_df = pd.merge(chart_one_df, music_info_df, on=["sing_id", "subject", "singer"])
chart_info_df

,date,rank,change,sing_id,subject,singer,rank_out_term,play_in_term,listen_in_term,play_total,listen_total,앨범 종류,발매일,앨범 장르,스타일,기획사,유통사,발매일_datetime
0,2019/12/08/11,1,-,89688529,십이월 이십오일의 고백,정승환,0,NaN,NaN,1939149,717398,싱글,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05
1,2019/12/08/12,1,-,89688529,십이월 이십오일의 고백,정승환,1,30490.0,6547.0,1969639,723945,싱글,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05
2,2019/12/08/13,1,-,89688529,십이월 이십오일의 고백,정승환,1,32874.0,7029.0,2002513,730974,싱글,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05
3,2019/12/08/14,1,-,89688529,십이월 이십오일의 고백,정승환,1,34436.0,7139.0,2036949,738113,싱글,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05
4,2019/12/08/15,1,-,89688529,십이월 이십오일의 고백,정승환,1,35033.0,7145.0,2071982,745258,싱글,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811,2021/01/04/15,224,NEW,89688529,십이월 이십오일의 고백,정승환,96,35781.0,1476.0,17589520,1511146,싱글,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05
2812,2021/01/05/9,227,NEW,89688529,십이월 이십오일의 고백,정승환,18,7471.0,377.0,17596991,1511523,싱글,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05
2813,2021/01/05/17,218,NEW,89688529,십이월 이십오일의 고백,정승환,8,4192.0,202.0,17601183,1511725,싱글,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05
2814,2021/01/06/9,238,NEW,89688529,십이월 이십오일의 고백,정승환,16,6208.0,263.0,17607391,1511988,싱글,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05


## 설명변수 생성

In [8]:
# date_time (str -> datetime 형식으로)
chart_info_df["date_time"] = chart_info_df["date"].apply(lambda x: datetime.strptime(x, "%Y/%m/%d/%H"))

# 날짜 설명변수 생성 (year, month, hour)
chart_info_df["date_time_year"] = chart_info_df["date_time"].dt.year
chart_info_df["date_time_month"] = chart_info_df["date_time"].dt.month
chart_info_df["date_time_hour"] = chart_info_df["date_time"].dt.hour
chart_info_df

,date,rank,change,sing_id,subject,singer,rank_out_term,play_in_term,listen_in_term,play_total,...,발매일,앨범 장르,스타일,기획사,유통사,발매일_datetime,date_time,date_time_year,date_time_month,date_time_hour
0,2019/12/08/11,1,-,89688529,십이월 이십오일의 고백,정승환,0,NaN,NaN,1939149,...,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05,2019-12-08 11:00:00,2019,12,11
1,2019/12/08/12,1,-,89688529,십이월 이십오일의 고백,정승환,1,30490.0,6547.0,1969639,...,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05,2019-12-08 12:00:00,2019,12,12
2,2019/12/08/13,1,-,89688529,십이월 이십오일의 고백,정승환,1,32874.0,7029.0,2002513,...,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05,2019-12-08 13:00:00,2019,12,13
3,2019/12/08/14,1,-,89688529,십이월 이십오일의 고백,정승환,1,34436.0,7139.0,2036949,...,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05,2019-12-08 14:00:00,2019,12,14
4,2019/12/08/15,1,-,89688529,십이월 이십오일의 고백,정승환,1,35033.0,7145.0,2071982,...,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05,2019-12-08 15:00:00,2019,12,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811,2021/01/04/15,224,NEW,89688529,십이월 이십오일의 고백,정승환,96,35781.0,1476.0,17589520,...,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05,2021-01-04 15:00:00,2021,1,15
2812,2021/01/05/9,227,NEW,89688529,십이월 이십오일의 고백,정승환,18,7471.0,377.0,17596991,...,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05,2021-01-05 09:00:00,2021,1,9
2813,2021/01/05/17,218,NEW,89688529,십이월 이십오일의 고백,정승환,8,4192.0,202.0,17601183,...,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05,2021-01-05 17:00:00,2021,1,17
2814,2021/01/06/9,238,NEW,89688529,십이월 이십오일의 고백,정승환,16,6208.0,263.0,17607391,...,2019.12.05,발라드,발라드,안테나,카카오엔터테인먼트,2019-12-05,2021-01-06 09:00:00,2021,1,9


In [9]:
# 시간 설명변수 생성 (발매일-현재)
chart_info_df["duration"] = chart_info_df["date_time"] - chart_info_df["발매일_datetime"]
chart_info_df["duration_days"] = chart_info_df["duration"].apply(lambda x: x.days)
chart_info_df["duration_seconds"] = chart_info_df["duration"].apply(lambda x: x.seconds)
chart_info_df["duration_hours"] = chart_info_df["duration_days"] * 24 + chart_info_df["duration_seconds"] // 3600
chart_info_df

,date,rank,change,sing_id,subject,singer,rank_out_term,play_in_term,listen_in_term,play_total,...,유통사,발매일_datetime,date_time,date_time_year,date_time_month,date_time_hour,duration,duration_days,duration_seconds,duration_hours
0,2019/12/08/11,1,-,89688529,십이월 이십오일의 고백,정승환,0,NaN,NaN,1939149,...,카카오엔터테인먼트,2019-12-05,2019-12-08 11:00:00,2019,12,11,3 days 11:00:00,3,39600,83
1,2019/12/08/12,1,-,89688529,십이월 이십오일의 고백,정승환,1,30490.0,6547.0,1969639,...,카카오엔터테인먼트,2019-12-05,2019-12-08 12:00:00,2019,12,12,3 days 12:00:00,3,43200,84
2,2019/12/08/13,1,-,89688529,십이월 이십오일의 고백,정승환,1,32874.0,7029.0,2002513,...,카카오엔터테인먼트,2019-12-05,2019-12-08 13:00:00,2019,12,13,3 days 13:00:00,3,46800,85
3,2019/12/08/14,1,-,89688529,십이월 이십오일의 고백,정승환,1,34436.0,7139.0,2036949,...,카카오엔터테인먼트,2019-12-05,2019-12-08 14:00:00,2019,12,14,3 days 14:00:00,3,50400,86
4,2019/12/08/15,1,-,89688529,십이월 이십오일의 고백,정승환,1,35033.0,7145.0,2071982,...,카카오엔터테인먼트,2019-12-05,2019-12-08 15:00:00,2019,12,15,3 days 15:00:00,3,54000,87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811,2021/01/04/15,224,NEW,89688529,십이월 이십오일의 고백,정승환,96,35781.0,1476.0,17589520,...,카카오엔터테인먼트,2019-12-05,2021-01-04 15:00:00,2021,1,15,396 days 15:00:00,396,54000,9519
2812,2021/01/05/9,227,NEW,89688529,십이월 이십오일의 고백,정승환,18,7471.0,377.0,17596991,...,카카오엔터테인먼트,2019-12-05,2021-01-05 09:00:00,2021,1,9,397 days 09:00:00,397,32400,9537
2813,2021/01/05/17,218,NEW,89688529,십이월 이십오일의 고백,정승환,8,4192.0,202.0,17601183,...,카카오엔터테인먼트,2019-12-05,2021-01-05 17:00:00,2021,1,17,397 days 17:00:00,397,61200,9545
2814,2021/01/06/9,238,NEW,89688529,십이월 이십오일의 고백,정승환,16,6208.0,263.0,17607391,...,카카오엔터테인먼트,2019-12-05,2021-01-06 09:00:00,2021,1,9,398 days 09:00:00,398,32400,9561


In [10]:
# 설명변수 One-Hot Encoding
music_type_col, chart_info_df = dummies_columns("앨범 종류", chart_info_df)
music_genre_col, chart_info_df = dummies_columns("앨범 장르", chart_info_df)
music_style_col, chart_info_df = dummies_columns("스타일", chart_info_df)

date_time_year_col, chart_info_df = dummies_columns("date_time_year", chart_info_df)
date_time_month_col, chart_info_df = dummies_columns("date_time_month", chart_info_df)
date_time_hour_col, chart_info_df = dummies_columns("date_time_hour", chart_info_df)
chart_info_df

,date,rank,change,sing_id,subject,singer,rank_out_term,play_in_term,listen_in_term,play_total,...,date_time_hour_14,date_time_hour_15,date_time_hour_16,date_time_hour_17,date_time_hour_18,date_time_hour_19,date_time_hour_20,date_time_hour_21,date_time_hour_22,date_time_hour_23
0,2019/12/08/11,1,-,89688529,십이월 이십오일의 고백,정승환,0,NaN,NaN,1939149,...,0,0,0,0,0,0,0,0,0,0
1,2019/12/08/12,1,-,89688529,십이월 이십오일의 고백,정승환,1,30490.0,6547.0,1969639,...,0,0,0,0,0,0,0,0,0,0
2,2019/12/08/13,1,-,89688529,십이월 이십오일의 고백,정승환,1,32874.0,7029.0,2002513,...,0,0,0,0,0,0,0,0,0,0
3,2019/12/08/14,1,-,89688529,십이월 이십오일의 고백,정승환,1,34436.0,7139.0,2036949,...,1,0,0,0,0,0,0,0,0,0
4,2019/12/08/15,1,-,89688529,십이월 이십오일의 고백,정승환,1,35033.0,7145.0,2071982,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811,2021/01/04/15,224,NEW,89688529,십이월 이십오일의 고백,정승환,96,35781.0,1476.0,17589520,...,0,1,0,0,0,0,0,0,0,0
2812,2021/01/05/9,227,NEW,89688529,십이월 이십오일의 고백,정승환,18,7471.0,377.0,17596991,...,0,0,0,0,0,0,0,0,0,0
2813,2021/01/05/17,218,NEW,89688529,십이월 이십오일의 고백,정승환,8,4192.0,202.0,17601183,...,0,0,0,1,0,0,0,0,0,0
2814,2021/01/06/9,238,NEW,89688529,십이월 이십오일의 고백,정승환,16,6208.0,263.0,17607391,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# One-Hot Encoding 추가
with open("../data/meta/columns_dict.json", "r") as f:
    columns_dict = json.load(f)

chart_info_df, music_type_columns = columns_add(columns_dict, "music_type_columns", music_type_col, chart_info_df)
chart_info_df, music_genre_columns = columns_add(columns_dict, "music_genre_columns", music_type_col, chart_info_df)
chart_info_df, music_style_columns = columns_add(columns_dict, "music_style_columns", music_type_col, chart_info_df)

chart_info_df, date_time_year_columns = columns_add(columns_dict, "date_time_year_columns", music_type_col, chart_info_df)
chart_info_df, date_time_month_columns = columns_add(columns_dict, "date_time_month_columns", music_type_col, chart_info_df)
chart_info_df, date_time_hour_columns = columns_add(columns_dict, "date_time_hour_columns", music_type_col, chart_info_df)
chart_info_df

,date,rank,change,sing_id,subject,singer,rank_out_term,play_in_term,listen_in_term,play_total,...,스타일_팝 락,스타일_어드벤처,스타일_락 발라드,스타일_TV 드라마,스타일_소울 팝,date_time_month_7,date_time_month_6,date_time_month_9,date_time_month_5,date_time_month_10
0,2019/12/08/11,1,-,89688529,십이월 이십오일의 고백,정승환,0,NaN,NaN,1939149,...,0,0,0,0,0,0,0,0,0,0
1,2019/12/08/12,1,-,89688529,십이월 이십오일의 고백,정승환,1,30490.0,6547.0,1969639,...,0,0,0,0,0,0,0,0,0,0
2,2019/12/08/13,1,-,89688529,십이월 이십오일의 고백,정승환,1,32874.0,7029.0,2002513,...,0,0,0,0,0,0,0,0,0,0
3,2019/12/08/14,1,-,89688529,십이월 이십오일의 고백,정승환,1,34436.0,7139.0,2036949,...,0,0,0,0,0,0,0,0,0,0
4,2019/12/08/15,1,-,89688529,십이월 이십오일의 고백,정승환,1,35033.0,7145.0,2071982,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811,2021/01/04/15,224,NEW,89688529,십이월 이십오일의 고백,정승환,96,35781.0,1476.0,17589520,...,0,0,0,0,0,0,0,0,0,0
2812,2021/01/05/9,227,NEW,89688529,십이월 이십오일의 고백,정승환,18,7471.0,377.0,17596991,...,0,0,0,0,0,0,0,0,0,0
2813,2021/01/05/17,218,NEW,89688529,십이월 이십오일의 고백,정승환,8,4192.0,202.0,17601183,...,0,0,0,0,0,0,0,0,0,0
2814,2021/01/06/9,238,NEW,89688529,십이월 이십오일의 고백,정승환,16,6208.0,263.0,17607391,...,0,0,0,0,0,0,0,0,0,0


## 데이터셋 생성

In [12]:
# 차트 인(rank_out_term) 유지 시간이 720시간 이상인 곡만 추림
sing_id_dict = {}

sing_id_list = chart_info_df["sing_id"].unique()

for s_idx, sing_id in enumerate(sing_id_list):
    compare_idx = -10
    compare_idx_list = []
    compare_tmp_list = []

    chart_one_df = chart_info_df[chart_info_df["sing_id"] == sing_id].reset_index(drop=True)
    df_index_list = chart_one_df[chart_one_df["rank_out_term"] == 1].index

    for e_idx, df_idx in enumerate(df_index_list):
        if compare_idx + 1 == df_idx:
            compare_tmp_list.append(df_idx)
        elif compare_idx != df_idx + 1 or e_idx == len(df_index_list) - 1:
            if len(compare_tmp_list) > 720:
                compare_idx_list = compare_idx_list + [compare_tmp_list]
            compare_tmp_list = []
        compare_idx = df_idx

    if compare_idx_list:
        sing_id_dict[sing_id] = compare_idx_list
        print("=" * 50)
        print(" sing_id: {}\n 제목: {}\n 가수: {}\n index cnt: {}".format(
            sing_id, chart_one_df.iloc[0]["subject"], chart_one_df.iloc[0]["singer"],
            [len(_) for _ in compare_idx_list]))
        print("=" * 50)

 sing_id: 89688529
 제목: 십이월 이십오일의 고백
 가수: 정승환
 index cnt: [728, 1119]


In [13]:
data_num = 30
meta_columns = music_type_columns + music_genre_columns + music_style_columns + \
               date_time_year_columns + date_time_month_columns + date_time_hour_columns + ["duration_hours"]

x_train_list = []
y_train_list = []

x_test_list = []
y_test_list = []

for sing_id, compare_idx_list in sing_id_dict.items():
    x_data_list = []
    y_data_list = []
    chart_one_df = chart_info_df[chart_info_df["sing_id"] == sing_id].reset_index(drop=True)

    for c_idx, compare_index in enumerate(compare_idx_list):
        chart_final_df = chart_one_df.iloc[compare_index[0]:compare_index[-1]].reset_index(drop=True)

        for i in range(0, chart_final_df.shape[0] - data_num):
            null_check_df = chart_final_df.iloc[i:i + data_num + 1][meta_columns + ["play_in_term"]]
            if null_check_df.isnull().values.any() or chart_final_df.iloc[i + data_num]["play_in_term"] == 0:
                continue

            x_data_meta = chart_final_df.iloc[i][meta_columns].values
            x_data_play = chart_final_df.iloc[i:i + data_num]["play_in_term"].values

            x_data_tmp = np.concatenate((x_data_meta, x_data_play))
            x_data_dims = np.expand_dims(x_data_tmp, axis=0)

            y_data_tmp = chart_final_df.iloc[i + data_num]["play_in_term"]
            y_data_dims = np.expand_dims(y_data_tmp, axis=0)

            x_data_list.append(x_data_dims)
            y_data_list.append(y_data_dims)

    data_cnt = int(len(x_data_list) * 0.8)
    x_train_list = x_train_list + x_data_list[:data_cnt]
    y_train_list = y_train_list + y_data_list[:data_cnt]

    x_test_list = x_test_list + x_data_list[data_cnt:]
    y_test_list = y_test_list + y_data_list[data_cnt:]

    print("=" * 50)
    print(" sing_id: {}\n 제목: {}\n 가수: {}\n Dataset count: {}".format(
        sing_id, chart_one_df.iloc[0]["subject"], chart_one_df.iloc[0]["singer"], len(x_data_list)))
    print("=" * 50)

 sing_id: 89688529
 제목: 십이월 이십오일의 고백
 가수: 정승환
 Dataset count: 1785


In [14]:
x_train = np.concatenate(x_train_list)
y_train = np.concatenate(y_train_list)

x_test = np.concatenate(x_test_list)
y_test = np.concatenate(y_test_list)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1428, 107), (1428,), (357, 107), (357,))

In [15]:
x_test

array([[0, 0, 0, ..., 1107.0, 853.0, 795.0],
       [0, 0, 0, ..., 853.0, 795.0, 680.0],
       [0, 0, 0, ..., 795.0, 680.0, 616.0],
       ...,
       [0, 0, 0, ..., 2400.0, 2080.0, 1767.0],
       [0, 0, 0, ..., 2080.0, 1767.0, 1525.0],
       [0, 0, 0, ..., 1767.0, 1525.0, 1330.0]], dtype=object)

In [16]:
y_test

array([ 680.,  616.,  613.,  864., 1495., 1928., 1568., 1720., 1903.,
       1968., 2195., 2266., 2299., 2358., 2578., 2765., 2387., 1972.,
       1647., 1579., 1299., 1028.,  899.,  809.,  723.,  599.,  685.,
        849., 1404., 2002., 1550., 1750., 1853., 1888., 2105., 2298.,
       2403., 2372., 2506., 2854., 2316., 1993., 1627., 1461., 1216.,
       1003.,  887.,  827.,  747.,  647.,  637.,  787., 1426., 1837.,
       1539., 1805., 1946., 1970., 2163., 2242., 2303., 2435., 2513.,
       2832., 2477., 2101., 1717., 1394., 1109.,  974.,  914.,  845.,
        754.,  698.,  636.,  693.,  868., 1094., 1408., 1723., 1962.,
       2108., 2273., 2320., 2472., 2395., 2432., 2145., 1960., 1715.,
       1526., 1396., 1149., 1025.,  897.,  811.,  749.,  663.,  677.,
        649.,  723.,  835., 1087., 1330., 1568., 1744., 1873., 1974.,
       2022., 1981., 2020., 1852., 1655., 1674., 1370., 1243., 1116.,
        936.,  798.,  749.,  616.,  551.,  694.,  807., 1495., 1862.,
       1566., 1554.,